# Dynamically creating classes

In [ ]:
Spam = type('Spam', (object,), dict(eggs='my eggs'))
spam = Spam()
print(spam.eggs)
print(type(spam))
print(type(Spam))

In [ ]:
class Spam(object, metaclass=type):
    pass

# A Basic metaclass

In [ ]:

class MetaSpam(type):
    def __new__(metaclass, name, bases, namespace):
        name = 'SpamCreatedByMeta'
        bases = (int, ) + bases
        namespace['eggs'] = 1
        return type.__new__(metaclass, name, bases, namespace)
    
class Spam(object, metaclass=MetaSpam):
    pass

print(Spam.__name__)
print(issubclass(Spam,int))
print(Spam.eggs)

# Arguments to metaclasses

In [ ]:

class MetaWithArguments(type):
    def __init__(metaclass, name, bases, namespace, **kwargs):
        type.__init__(metaclass, name, bases, namespace)
        
    def __new__(metaclass, name, bases, namespace, **kwargs):
        for k, v in kwargs.items():
            namespace.setdefault(k, v)
        return type.__new__(metaclass, name, bases, namespace)

class WithArgument(metaclass=MetaWithArguments, spam='eggs'):
    pass

with_argument = WithArgument
print(with_argument.spam)

# Accessing metaclass attributes through classes

In [ ]:

class Meta(type):
    
    @property
    def spam(cls):
        return 'Spam property of %r' % cls 
    
    def eggs(self):
        return 'Eggs method of %r ' % self

class SomeClass(metaclass=Meta):
    pass

print(SomeClass.spam)
print(SomeClass.eggs)
print(SomeClass().spam)

# Abstract classes using collections.abc

In [ ]:
import abc

class Spam(metaclass=abc.ABCMeta):
    
    @abc.abstractmethod
    def some_method(self):
        raise NotImplemented()

class Eggs(Spam):
    def some_new_method():
        pass
    
eggs = Eggs()


In [4]:
# emulate abc.ABCMeta

import functools

class AbstractMeta(type):
    def __new__(metaclass, name, bases, namespace):
        cls = super().__new__(metaclass, name, bases, namespace)
        
        # collect all local methods marked as abstract
        abstracts = set()
        for k, v in namespace.items():
            if getattr(v, '__abstract__', False):
                abstracts.add(k)
                
        # Look for abstract methods in the base classes
        for base in bases:
            for k in getattr(base, '__abstracts__', ()):
                v = getattr(cls, k, None)
                if getattr(v, '__abstract__', False):
                    abstracts.add(k)
        
        # Store the abstracts in a frozenset so they cannot be modified
        cls.__abstracts__ = frozenset(abstracts)
        
        # Decorate the __new__ function to check if all abstract
        # functions were implemented
        original_new = cls.__new__
        @functools.wraps(original_new)
        def new(self, *arg, **kwargs):
            for k in self.__abstracts__:
                v = getattr(self, k)
                if getattr(v, '__abstract__', False):
                    raise RuntimeError( '%r is not really implemented' % k)
            return original_new(self, *args, **kwargs)
        cls.__new__ = new
        return cls
    
def abstractmethod(function):
    function.__abstract__ = True
    return function

class Spam(object, metaclass=AbstractMeta):
    @abstractmethod
    def some_method(self):
        pass

eggs = Spam()

RuntimeError: 'some_method' is not really implemented

# Custom type checks

In [9]:
import abc

class CustomList(abc.ABC):
    pass

print(CustomList.register(list))
print(issubclass(list, CustomList))
print(isinstance([], CustomList))
print(issubclass(CustomList, list))
print(isinstance(CustomList, list))

<class 'list'>
True
True
False
False
